In [1]:
import struct

In [2]:
def to_csv(name, maxdata):
    lbl_f = open(f"mnist/{name}-labels.idx1-ubyte","rb")
    img_f = open(f"mnist/{name}-images.idx3-ubyte","rb")
    csv_f = open(f"mnist/{name}-.csv",'w', encoding="utf-8")
    mag, lbl_count = struct.unpack('>II', lbl_f.read(8))
    mag, img_count = struct.unpack('>II', img_f.read(8))
    rows, cols = struct.unpack('>II', img_f.read(8))
    pixels = rows * cols
    
    res = []
    for idx in range(lbl_count):
        if idx > maxdata: break;
        label = struct.unpack("B", lbl_f.read(1))[0]
        bdata = img_f.read(pixels)
        sdata = list(map(lambda n: str(n), bdata))
        csv_f.write(str(label)+",")
        csv_f.write(",".join(sdata)+"\r\n")
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = f"./mnist/{name}-{idx}-{label}.pgm"
            with open(iname, 'w', encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()

In [3]:
to_csv("train", 1000)
to_csv("t10k", 500)

In [6]:
def load_csv(fname):
    labels = []
    images = []
    with open(fname, 'r') as f:
        for line in f:
            cols = line.split(',')
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n:int(n) / 256, cols))
            images.append(vals)
    return {'labels':labels, 'images':images}
data = load_csv('./mnist/train-.csv')
test = load_csv('./mnist/t10k-.csv')

In [10]:
from sklearn import model_selection, svm, metrics

C:\ProgramData\Anaconda3\envs\collect\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [12]:
clf = svm.SVC()
clf.fit(data['images'], data['labels'])
predict = clf.predict(test['images'])

In [13]:
ac_score = metrics.accuracy_score(test['labels'], predict)
cl_report = metrics.classification_report(test['labels'], predict)
print(f'정답률 = {ac_score}')
print('리포트')
print(cl_report)

정답률 = 0.8842315369261478
리포트
              precision    recall  f1-score   support

           0       0.87      0.98      0.92        42
           1       0.99      1.00      0.99        67
           2       0.91      0.89      0.90        55
           3       0.94      0.72      0.81        46
           4       0.86      0.93      0.89        55
           5       0.75      0.82      0.78        50
           6       0.95      0.81      0.88        43
           7       0.79      0.94      0.86        49
           8       0.94      0.82      0.88        40
           9       0.89      0.87      0.88        54

    accuracy                           0.88       501
   macro avg       0.89      0.88      0.88       501
weighted avg       0.89      0.88      0.88       501

